<a href="https://colab.research.google.com/github/BankNatchapol/ML-Algorithm/blob/master/Normal_Equation_vs_Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Linear Regression: Normal Equation vs Gradient Descent**
---
ในครั้งที่แล้วผมได้ทำ Multiple Variable Linear Regression ไปแล้วซึ่งในคราวที่แล้วการ Update parameters ในแต่ละครั้งนั้นใช้Gradient Descentเป็นหลัก <br>
ในครั้งนี้จะมาลองใช้อีก Algorithm หนึ่งที่เรียกว่า **Normal Equation**<br><br>
**ideas** : เมื่อกำหนดให้ $\theta^T$ คือ vector <$\theta _0 , \theta _1,\theta _2 , \theta_3,...,\theta_n$>, x คือ vector <$x_0 , x_1,x_2 , x_3,...,x_n$>
<br>จะได้<br>
\begin{equation}
\theta x  = y
\end{equation}
<br>
ซึ่งถ้าเราอยากหา $\theta^T$ ก็สามารถทำได้โดยการย้ายข้างสมการง่ายๆ<br><br>
\begin{equation}
\theta  = x^{-1}y
\end{equation}
<br>
แต่ในความเป็นจริงแล้วเราไม่สามารถทำแบบนี้ได้เนื่องจาก x เป็น Matrix และ การ inverse ไม่ได้ทำได้กับทุก Matrix <br>ดังนั้นเราจึงคูณทั้ง 2 ข้างของสมการด้วย Matrix$\;x^T $ <br>
\begin{equation}
\theta x^Tx  = x^Ty 
\end{equation}<br>
และทำการ inverse $x^Tx $ โดยที่จะเป็นการทำ inverse แบบ Psudo-inverse เพื่อให้สามารถหา inverse ของ Matrix ได้ทุกแบบ <br><br>
\begin{equation}
\theta  = (x^Tx)^{-1}x^Ty 
\end{equation}<br>
ดังนั้นเราก็จะสามารถ update parameters ได้แล้ว ด้วยสมการ Normal Equation

In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
pd.set_option('display.max_rows', df.shape[0]+1) 

โดยขั้นแรกจะทำการโหลด boston housing datasets ด้วย library sklearn 

In [0]:
dat = load_boston(return_X_y=False) # boston housing datasets

In [0]:
df = pd.DataFrame(dat.data,columns=dat.feature_names) # create dataframe
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


เป้าหมายของเราคือการทำนายราคาของบ้าน จากข้อมูล

In [0]:
target = pd.DataFrame(dat.target.T,columns=['PRICES']) # target prices
target.head()

,PRICES
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [0]:
def scaling(x): #scaling input 
  std = np.std(x)
  std[0] = 1
  mean = np.mean(x)
  mean[0] = 0
  return (x-mean)/std

df_s = scaling(df)
ones = pd.DataFrame({'ones':np.ones(df_s.shape[0]).T}) 
df_s = ones.join(df_s)
m = df_s.shape[0]
n = df_s.shape[1]
s = np.zeros(n)
sT = np.array([s]).T #all parameters

def hypo(x,sT): #hypothesis function
    return np.dot(x,sT)

def costFunction(x,sT): #cost function
    h = hypo(x,sT)
    return (1/(2*m))*sum(np.array((h-target))**2)
  
def gradientCost(x,sT): #gradient descent
    h = hypo(x,sT)
    return (1/(m))*np.dot(df_s.T,(h-target))

alpha = 0.01 # learning rate
last = costFunction(df_s,sT)+1
while last - costFunction(df_s,sT)>0.001: #loop until change < 0.001
  last = costFunction(df_s,sT)
  sT = sT - alpha*(gradientCost(df_s,sT))
GDpredicted = np.dot(df_s,sT)
compare = pd.DataFrame({'Target Prices':np.array(target).T[0],'GD Predicted Prices':GDpredicted.T[0]}) # comparing result with target 
compare.head()

,Target Prices,GD Predicted Prices
0,24.0,30.595625
1,21.6,24.982998
2,34.7,30.979016
3,33.4,29.284144
4,36.2,28.673260


ใช้ function pinv ของ numpy ในการทำ Matrix psudo-inverse 

In [0]:
from numpy.linalg import inv,pinv #import การทำ psudo-inverse

ทำการคำนนวนหา parameters ตามสมการ <br>
\begin{equation}
\theta  = (x^Tx)^{-1}x^Ty 
\end{equation}<br>

In [0]:
df_n = df
a = pinv(np.dot(df_n.T,df_n)) # psudo inverse
b = np.dot(df_n.T,target) 
theta = np.dot(a,b) #parameters
theta

array([[-9.28965170e-02],
       [ 4.87149552e-02],
       [-4.05997958e-03],
       [ 2.85399882e+00],
       [-2.86843637e+00],
       [ 5.92814778e+00],
       [-7.26933458e-03],
       [-9.68514157e-01],
       [ 1.71151128e-01],
       [-9.39621540e-03],
       [-3.92190926e-01],
       [ 1.49056102e-02],
       [-4.16304471e-01]])

ทำการเปรียบเทียบ Predicted Prices	ที่ได้จาก Gradient Descent และจาก Normal Equation

In [0]:
NEpredicted = np.dot(df_n,theta) #predict using normal equation's parameters
sh = pd.DataFrame({'Target Prices':np.array(target).T[0],'GD Predicted Prices':np.dot(df_s,sT).T[0],'NE Predicted Prices':NEpredicted.T[0]})
sh.head()

,Target Prices,GD Predicted Prices,NE Predicted Prices
0,24.0,30.595625,29.098264
1,21.6,24.982998,24.502275
2,34.7,30.979016,31.227426
3,33.4,29.284144,29.707104
4,36.2,28.673260,29.564796


หลังจากเปรียบเทียบค่าต่อค่าไปแล้วก็ลองมาดูความแม่นยำจาก RMSE ของทั้ง 2 ผลลัพท์

In [0]:
def rmse(p):
  return np.sqrt(sum((np.array(target) - p)**2)/len(p))

In [0]:
print(f'RMSE of GD is {rmse(GDpredicted)[0]}\nRMSE of NE is {rmse(NEpredicted)[0]}')

RMSE of GD is 4.757750208938539
RMSE of NE is 4.915902697381884


จะพบได้ว่า มีผลลัพท์ที่ใกล้เคียงกัน เนื่องจากมีจำนวน feature กับ example น้อยจึงอาจจะไม่เห็นความแตกต่างมากเท่าไร <br>
ซึ่งในการเปรียบเทียบข้อดีข้อเสียของทั้ง 2 Algorithm นั้น <br>

>**Gradient Descent**<br>

>ข้อดี|ข้อเสีย
>--- | ---
>-สามารถใช้งานกับ feature จำนวนมากได้  $O(kn^2)$| -จำเป็นที่จะต้องทำการเลือก $\alpha$ ซึ่งอาจจะไม่ได้ $\alpha$ ที่ดีที่สุด
>  | -จำเป็นที่จะต้องใช้ iteration หลายรอบซึ่งทำให้ช้าลง

>**Normal Equation**<br>

>ข้อดี|ข้อเสีย
>--- | ---
>-ไม่จำเป็นที่จะต้องเลือก $\alpha$| -ต้องทำการคำนวน $x^Tx$ ซึ่งเป็น matrix ขนาด nxn <br> ทำให้เมื่อมี feature มากจะต้องใช้เวลามาก $O(n^3)$
>-ไม่มี iteration เนื่องจาก เป็นการ dot ครั้งเดียว |   

<br>